# 使用 mallet 训练 LDA的示例代码

* * *
### 原始文件
**raw.txt**: 原始数据文件。每一行是一篇文章，已经经过了分词处理。

In [11]:
!head -n 6 raw.txt | awk '{ print $2" "$3" "$4" "$5" "$6" "$7" "$8" "$9" "$10" "$11"...." }'

： 海清 密会 小沈阳 聚餐 男方 当街 提 裤子 ,....
大越野 ss2 ： 赛车 再 拖后腿 魏红杰 深谋远虑 , 　....
年 国军 冬季 大 反攻 蒋公 总结 , 蒋介石 关于....
姐 麦明诗 不在意 走 光 整容 说 : 牙 都....
： 创业板 或 逐步 进入 业绩 兑现 期 , [....
学 摄影 ： 用 ps 打造出 手绘 特效 照片 ,....


* * *
### 字典文件
**words.dict**: 有效词的词典。 格式： id word other-info 

In [12]:
!head -n 6 words.dict

1`记者`302819
2`中国`262746
3`情况`243587
4`今年`235291
5`问题`231584
6`时间`230710


* * *
###  mallet的输入文本
将原始文件转换成mallet能接受的输入文本: **examples.data**

输入文本的格式有[详细的说明](http://mallet.cs.umass.edu/import.php):
>the `first` token of each line (whitespace delimited, with optional comma) becomes the instance name, the `second` token becomes the label, and `all additional` text on the line is interpreted as a sequence of word tokens.

需要注意的是，如果用于LDA的输入文本，必须是**a sequence of word tokens**而不是**a vector of feature/value pairs**:
>--keep-sequence. This option preserves the document as a sequence of word features, rather than a vector of word feature counts. Use this option for sequence labeling tasks. The MALLET topic modeling toolkit also requires feature sequences rather than feature vectors.

为了避免中文可能导致的问题，我将所有的词都转换成了对应的词id。

In [13]:
!head -n 2 examples.data

1 lab 2136 48210 75483 46484 14675 12958 24227 1907 6463 826 492 6808 83139 46484 5227 4142 569 7226 106102 1158 11826 4210 24856 1245 228684 50118 1 56848 6025 1766 13723 46484 48210 5502 3224 1611 1379 121 32784 1571 46484 1379 3856 1472 38953 2734 14206 42664 1537 1537 1907 6463 1027 3856 2353 3598 10533 1527 1006 3856 7409 1 177 2674 3856 2752 552 12331 3856 1217 2142 46484 2734 6025 15209 48210 38953 2734 25623 48210 204 5513 5269 3865 8986 533 3817 1632 3759 6775 310 21 6025 428 1379 34144 14675 143 2349 48210 348 64 577 26926 80932 463 53212 3887 14684 121 561 463 48210 26926 80932 986 71 3291 161 463 25515 2846 284 1033 2620 109746 333 2374 463 3402 1029 9276 25515 52893 2846 7016 2620 4602 333 19733 8347 7668 7927 1865 41088 8916 48210 3533 2872 31969 46484 12011 2820 47932 5161 3555 139 311 216966 139 3028 3606 9603 5615 374 7438 4821 14324 11834 3555 44 5787 1123 1664 3310 1402 1019 20545 2825 3187 676 7170 9603 10330 1663 1664 47932 1757 3114 1379 2960 20047 5615 15483 2501

* * *
### 用mallet导入数据
使用`mallet import-file`导入输入文本。

可以使用`mallet import-file --help`查看详细的使用方法。

生成的**examples.mallet**是一个二进制文件。

In [1]:
!mallet import-file --input examples.data --output examples.mallet --keep-sequence --token-regex '[a-z0-9]+'

* * *
### LDA训练
使用`mallet train-topics`进行LDA训练。

使用`mallet train-topics --help`查看详细的使用方法。

In [ ]:
!mallet train-topics --input examples.mallet --num-topics 50 --num-threads 1 --num-iterations 800 \
        --optimize-burn-in 200 --alpha 5 --beta 0.01 --optimize-interval 10 --output-state examples-topic-state.gz \
        --output-topic-keys examples_keys.txt --output-doc-topics examples_composition.txt --show-topics-interval 100

* * *
### LDA训练的结果

####  Topic的效果 `examples_keys.txt`

**topic-keys**文件中保存的是每个topic排名最前的词:
> The filename in which to write the top words for each topic and any Dirichlet parameters.
> The second number in each paragraph is the Dirichlet parameter for the topic.

In [3]:
!head -n 5 examples_keys.txt

0	0.04458	55 60 2401 547 2118 147 395 94 1242 152 330 1025 1202 6536 994 1381 467 4470 3031 2218 
1	0.04353	1586 518 2799 126 690 1375 1536 2297 4157 2025 54 527 3799 955 421 3623 2070 53 6414 1803 
2	0.06352	32 305 20 496 309 2 55 393 12 14 85 112 114 129 303 642 45 6293 402 573 
3	0.02874	5014 122 382 8804 2098 2062 2 8116 1390 267 7330 2555 9533 6490 1009 2219 4119 328 532 6081 
4	0.02526	1406 11187 1109 36 237 21362 1690 1 13169 5774 12534 10171 9 3453 303 22811 9571 7991 2191 180 


这样看着不清楚，我需要将里面的id转换成词:

In [14]:
import pandas as pd
import codecs

def load_vec_dict(vfile):
    vdict = {}
    for line in codecs.open(vfile, "r", "utf8"):
        line = line.strip()
        items = line.split("`")
        if len(items) < 2:
            continue
        vdict[int(items[0])] = items[1]
    return vdict

def print_topic_keys(tkfile, vdict):
    for line in codecs.open(tkfile, "r", "utf8"):
        items = line.split("\t")
        topicid = items[0]
        ids = items[2].split(" ")
        ids = ids[:-1]
        words = [vdict.get(int(id)) for id in ids]
        print topicid + " : " + " ".join(words)

vdict = load_vec_dict("words.dict")
print_topic_keys("examples_keys.txt", vdict)

0 : 项目 建设 垃圾 工程 公园 城市 规划 环境 设施 中心 市民 生态 建筑 搬迁 改造 环保 道路 停车场 西安 城区
1 : 事故 车辆 交警 安全 交通 驾驶 违法 运输 机动车 执法 发生 检查 爆炸 民警 有限公司 违法行为 整治 现场 火灾 天津
2 : 企业 集团 公司 改革 产业 中国 项目 开发 市场 发展 技术 行业 计划 领域 生产 转型 未来 天然气 我国 运营
3 : 抗战 历史 日本 日军 周年 战争 中国 抗日 胜利 上海 抗日战争 纪念 抗战胜利 神话 老人 和平 战场 故事 当年 中国人民
4 : 食品 暴雨 天气 部分 地区 检出 检测 记者 积水 毫米 样品 大雨 出现 气温 生产 微商 批次 雨水 监测 严重
5 : 工作 服务 建设 开展 推进 政府 农村 保障 社会 社区 创业 实施 记者 建立 资金 重点 改革 发展 就业 完善
6 : 平台 互联网 服务 行业 市场 用户 模式 数据 提供 融资 产品 公司 企业 投资 领域 创业 电商 运营 客户 钉
7 : 规定 信息 记者 相关 要求 服务 使用 办理 根据 发布 不得 情况 登记 广告 企业 管理 机构 单位 部门 其他
8 : 现在 我的 大家 时间 起来 喜欢 最后 今天 朋友 事情 真的 后来 感觉 希望 工作 于是 出来 一起 当然 曾经
9 : border td solid width top 1px 0px none 7px style padding left right bottom valign pt tr 王迅 资料 color
10 : 周边 地铁 公交 车辆 记者 线路 交通 道路 出行 配套 公里 泰禾 乘客 市民 施工 号线 方向 公交车 时间 通行
11 : 科学家 火星 细节 技术 李晨 研究 千足虫 地球 月球 人类 来自 敦煌 碎叶 余生 壁画 物种 范冰冰 方法 报道 ct
12 : 拍摄 音乐 摄影 照片 侯孝贤 聂隐娘 电影 作品 拍 镜头 梅毒 故事 the 人物 摄影师 周韵 歌手 风格 少年 女人
13 : 手机 支持 三星 设计 屏幕 接口 产品 像素 采用 功能 处理器 机身 方面 nbsp galaxy 系列 英寸 版 性能 摄像头
14 : 中国 美国 日本 报道 俄罗斯 国家 政府 英国 全球 

#### 每篇文章关于Topic的分布 `examples_composition.txt`

**doc-topics**保存的是每篇文章的Topic分布:
> The filename in which to write the topic proportions per document, at the end of the iterations.

In [16]:
!head -n 1 examples_composition.txt

0	1	1.8710862416137548E-4	1.8268029448505129E-4	2.6660203127081136E-4	1.2061353446059575E-4	1.0600426781484946E-4	4.3220776934051777E-4	1.9949368676932172E-4	4.458226657652823E-4	0.21492698848282757	1.0074355283481964E-5	1.8906709220776387E-4	5.164549796542017E-5	1.0599858535676162E-4	9.250077632081983E-5	2.1863023532950842E-4	1.151840866576433E-4	2.1879961957341882E-4	3.392818899873787E-5	0.059143678021967815	6.76591914659558E-5	1.6336606230725803E-4	3.3422424544119755E-5	0.08011125397212848	7.995486796415965E-5	8.78745229395449E-5	1.5915748457406072E-4	0.01309693753539036	8.695321730618228E-5	8.269894375319518E-5	2.1182513956066311E-4	1.3241156671328898E-4	0.03376089984861472	4.463920307860546E-5	0.13443187266673162	3.871803131338747E-5	1.5133103634966806E-4	8.296260231016834E-5	3.3164326943691246E-5	1.1383334115769523E-4	4.593339137625585E-5	7.60248674107469E-5	6.674010863701453E-5	0.2728985423208539	1.1521798938855096E-4	7.108728108218255E-5	1.317858624565444E-4	0.06315400163890597